In [ ]:
%pip install xgboost
%pip install tensorflow

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
import sys
sys.path.append("../utils")
import joblib

from utils import load_csv
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
symbols = ["fpt", "hpg", "vnm"]
dfs = {s: load_csv(f"../../data/features/{s}_features.csv") for s in symbols}
feature_cols = [
    "open", "high", "low", "close", "volume",
    "return", "ma5", "ma10", "ma20", "volatility_5", "vol_change"
]

2025-05-27 19:38:39,070 - INFO - ✅ Load CSV thành công: ../../data/features/fpt_features.csv | shape = (4521, 13)
2025-05-27 19:38:39,087 - INFO - ✅ Load CSV thành công: ../../data/features/hpg_features.csv | shape = (4318, 13)
2025-05-27 19:38:39,103 - INFO - ✅ Load CSV thành công: ../../data/features/vnm_features.csv | shape = (4753, 13)


In [19]:
df = load_csv("../../data/features/fpt_features.csv")

2025-05-27 19:49:00,658 - INFO - ✅ Load CSV thành công: ../../data/features/fpt_features.csv | shape = (4521, 13)


In [20]:
df.index = df.date
df.drop('date',axis=1,inplace=True)
df

,open,high,low,close,volume,EMA_9,SMA_5,SMA_10,SMA_15,SMA_30,RSI_14,MACD
date,,,,,,,,,,,,
2006-12-13,11.44,11.44,11.44,11.44,83530,13.137137,13.266,13.273,13.235333,14.397667,40.920716,-0.847831
2006-12-14,12.01,12.01,12.01,12.01,280710,13.418922,13.636,13.444,13.378000,14.550000,40.609137,-0.755893
2006-12-15,12.61,12.61,12.61,12.61,265300,13.621152,13.750,13.498,13.471333,14.706333,40.302267,-0.685588
2006-12-18,13.24,13.24,13.24,13.24,215790,13.716440,13.606,13.432,13.522667,14.870000,44.832215,-0.646972
2006-12-19,13.89,13.89,13.89,13.89,137520,13.673050,13.452,13.329,13.540000,15.021667,49.117647,-0.652419
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-07,141.70,142.70,141.10,141.40,2954500,127.966186,133.540,123.110,119.200000,117.700000,85.507246,5.815931
2025-03-10,141.50,141.50,139.20,139.30,5323000,125.132732,129.380,120.350,117.213333,116.950000,83.879093,4.897142
2025-03-11,138.00,138.50,137.00,137.50,5784400,122.040915,125.080,117.640,115.340000,116.270000,84.313725,3.867020


In [21]:
df1 = df.reset_index()['close']
df1.head()

0    11.44
1    12.01
2    12.61
3    13.24
4    13.89
Name: close, dtype: float64

In [22]:
# chuẩn hoá dữ liệu
scaler = MinMaxScaler(feature_range=(0, 1))
df1 = scaler.fit_transform(np.array(df1).reshape(-1,1))
joblib.dump(scaler, "../../model/fpt_scaler.pkl")

['../../model/fpt_scaler.pkl']

In [23]:
split_index = int(len(df1)*0.7)
train_data, test_data = df1[:split_index], df1[split_index:]
train_data

array([[0.06130495],
       [0.06505027],
       [0.06899271],
       ...,
       [0.13272883],
       [0.13384585],
       [0.13548853]])

In [24]:
def create_dataset(dataset, time_step = 50):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step - 1):
        dataX.append(dataset[i:(i+time_step),0])
        dataY.append(dataset[i+time_step, 0])
    return np.array(dataX), np.array(dataY)

In [25]:
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

In [26]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [27]:
# Khởi tạo mô hình
model = Sequential()

# Lớp 1
model.add(LSTM(units=128, return_sequences=True, input_shape=(100,1)))
model.add(Dropout(0.2))
# Lớp 2
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))
# Lớp 3
model.add(LSTM(units=32))
model.add(Dropout(0.2))
# Lớp đầu ra
model.add(Dense(1))

# Compile mô hình với thuật toán tối ưu adam và đo hàm mất mát MAE
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()


c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 100, 128)       │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 100, 64)        │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 100, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 128,417 (501.63 KB)

 Trainable params: 128,417 (501.63 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
save_model_path = "../../model/fpt_lstm_model.keras"

# Lưu mô hình khi val_loss tốt hơn
best_model = ModelCheckpoint(
    filepath=save_model_path,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min'
)

# Dừng sớm nếu val_loss không cải thiện trong 20 epoch
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=64,
    verbose = 1,
    validation_data=(X_test, y_test),
    callbacks=[best_model, early_stop]
)

Epoch 1/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - loss: 3.5730e-05
Epoch 1: val_loss improved from inf to 0.00051, saving model to ../../model/fpt_lstm_model.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 118ms/step - loss: 3.5678e-05 - val_loss: 5.0667e-04
Epoch 2/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - loss: 3.0627e-05
Epoch 2: val_loss did not improve from 0.00051
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 123ms/step - loss: 3.0619e-05 - val_loss: 7.5110e-04
Epoch 3/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - loss: 2.8815e-05
Epoch 3: val_loss improved from 0.00051 to 0.00045, saving model to ../../model/fpt_lstm_model.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 8s 161ms/step - loss: 2.8813e-05 - val_loss: 4.5294e-04
Epoch 4/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - loss: 2.8577e-05
Epoch 4: val_loss improved from 0.00045 to 0.00043, saving model to ../../model/fpt_lstm_model.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 11s 234ms/step - loss: 2.8573e-05 - val_loss: 4.2835e-04
Epoch 5/100
48/48 ━━━━━━━━━━━━━━━

In [ ]:
final_model = load_model("../../model/fpt_lstm_model.keras")
train_predict = final_model.predict(X_train)
test_predict = final_model.predict(X_test)

92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step


In [ ]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

In [ ]:
### Calculate RMSE performance metrics
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(y_train,train_predict))

5.614266503544013

In [ ]:
### Test Data RMSE
math.sqrt(mean_squared_error(y_test,test_predict))

24.411810436433214

In [ ]:
look_back=100
trainPredictPlot = np.empty_like(df1)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
dates = df.index

testPredictPlot = np.empty_like(df1)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(df1)-1, :] = test_predict
plt.figure(figsize=(14,5))
plt.plot(scaler.inverse_transform(df1), color='b', label='Original Data')
plt.plot(trainPredictPlot, color='g', label='Train Predict')
plt.plot(testPredictPlot, color='r', label='Test Predict')
plt.ylabel('Close Price')
plt.title('Stock Price Prediction')
plt.legend()
plt.tight_layout()
plt.show()